
# SQL Agent
- LangChain은 자연어 프롬프트를 기반으로 SQL 쿼리를 작성하고 실행할 수 있는 SQL 체인 및 에이전트를 제공
- SQLAlchemy에서 지원하는 모든 SQL 언어(예: MySQL, PostgreSQL, Oracle SQL, Databricks, SQLite)와 호환

In [4]:
!pip install langchain langchain-experimental openai

import os
import openai
os.environ["OPENAI_API_KEY"] = 'api_key'

Chinook DB와의 SQLite 연결을 수행합니다.

* [이 파일](https://www.sqlitetutorial.net/sqlite-sample-database/)을 디렉터리에 `Chinook_Sqlite.sql`로 저장
* `sqlite3 Chinook.db`를 실행
* `.read Chinook_Sqlite.sql` 실행
* 테스트 `SELECT * FROM Artist LIMIT 10;`

SQL 쿼리를 생성하고 실행하기 위해 `SQLDatabaseChain`을 생성

In [5]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [6]:
db_chain.run("몇명의 직원이 있어?")



> Entering new SQLDatabaseChain chain...
몇명의 직원이 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

### (1) Text-to-SQL query


In [7]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI

SQL 쿼리를 작성할 체인 생성

In [8]:
chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


SELECT COUNT("EmployeeId") AS "TotalEmployees" FROM employees


사용자 질문을 기반으로 SQL 쿼리를 작성하고 나면 쿼리 실행 가능

In [9]:
db.run(response)

'[(8,)]'

SQL 쿼리 빌더 체인은 쿼리를 생성만 하고 쿼리 실행은 별도로 처리

### (3) Text-to-SQL query and execution

- SQL 쿼리를 작성하고 실행하기 위해 `langchain_experimental`의 `SQLDatabaseChain` 사용

In [10]:
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [11]:
db_chain.run("직원이 몇명이나 있어?")



> Entering new SQLDatabaseChain chain...
직원이 몇명이나 있어?
SQLQuery:SELECT COUNT(*) FROM employees
SQLResult: [(8,)]
Answer:8
> Finished chain.


'8'

(1)번 예제와 동일한 결과

여기서 체인은 쿼리 실행도 처리하고 사용자 질문과 쿼리 결과를 기반으로 최종 답변 제공

이 방식은 'SQL 인젝션'에 취약하기 때문에 사용 시 주의 필요

* 체인이 LLM에 의해 생성되고 검증되지 않은 쿼리를 실행할 가능성 있음
* 예: 레코드가 의도치 않게 생성, 수정 또는 삭제될 수 있음

## (3) SQL 에이전트
- SQL 에이전트: SQLDatabaseChain`보다 SQL 데이터베이스와 상호 작용하는 더 유연한 방법을 제공
- 데이터베이스의 스키마뿐만 아니라 데이터베이스의 콘텐츠(예: 특정 테이블 설명)를 기반으로 질문에 답변 가능
- 생성된 쿼리를 실행하고 트레이스백을 포착하여 올바르게 다시 생성함으로써 오류로부터 복구 가능

- 에이전트를 초기화하기 위해 `create_sql_agent` 함수 사용

- 이 에이전트에는 다음과 같은 도구가 포함된 `SQLDatabaseToolkit`이 포함

  * 쿼리 생성 및 실행
  * 쿼리 구문 확인
  * 테이블 설명 검색
  * ... 등

In [12]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///chinook.db")

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

### Agent task example #1 - Running queries


In [13]:
agent_executor.run(
    "국가별 총 매출을 나열합니다. 어느 국가의 고객이 가장 많이 지출했나요?"
)



> Entering new SQL Agent Executor chain...
 We need to find the total sales for each country and then determine which country has the highest total sales.
Action: sql_db_query
Action Input: SELECT country, SUM(sales) AS total_sales FROM sales_table GROUP BY country ORDER BY total_sales DESC LIMIT 1Error: (sqlite3.OperationalError) no such table: sales_table
[SQL: SELECT country, SUM(sales) AS total_sales FROM sales_table GROUP BY country ORDER BY total_sales DESC LIMIT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)We need to check the table name and make sure it exists in the database.
Action: sql_db_list_tables
Action Input: ""albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks The sales_table is not listed, so we need to use sql_db_schema to get the correct table fields.
Action: sql_db_schema
Action Input: sales_tableError: table_names {'sales_table'} not found in database We need to check the spell

'The USA has the highest total sales.'

### Agent task example #2 - Describing a Table

In [14]:
agent_executor.run("playlisttrack 테이블에 대해서 설명해줄래?")



> Entering new SQL Agent Executor chain...
 I should use sql_db_schema to get the schema and sample rows for the playlisttrack table.
Action: sql_db_schema
Action Input: playlisttrackError: table_names {'playlisttrack'} not found in databaseI should use sql_db_list_tables to get a list of tables in the database and check if playlisttrack is included.
Action: sql_db_list_tables
Action Input: ""albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracksI should use sql_db_schema again with the correct spelling of playlist_track.
Action: sql_db_schema
Action Input: playlist_track
CREATE TABLE playlist_track (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES tracks ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES playlists ("PlaylistId")
)

/*
3 rows from playlist_track table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/I now know the final

'The playlisttrack table has two columns, PlaylistId and TrackId, and is used to store the relationship between playlists and tracks. It has a composite primary key consisting of both columns and foreign key constraints referencing the playlists and tracks tables. Sample rows from the table show the relationship between a playlist with id 1 and three tracks with ids 3402, 3389, and 3390.'

### (4) SQL 툴킷 확장하기

- SQL 툴킷 확장: 솔루션의 전반적인 성능을 개선하기 위해 솔루션에서 도메인별 지식을 사용하려고 할 때 특히 유용
- 예시
  - Dynamic Few shot 예시 포함
  - 열 필터로 사용할 고유명사의 철자 오류 찾기


질문 목록 생성

In [15]:
few_shots = {
    "List all artists.": "SELECT * FROM artists;",
    "Find all albums for the artist 'AC/DC'.": "SELECT * FROM albums WHERE ArtistId = (SELECT ArtistId FROM artists WHERE Name = 'AC/DC');",
    "List all tracks in the 'Rock' genre.": "SELECT * FROM tracks WHERE GenreId = (SELECT GenreId FROM genres WHERE Name = 'Rock');",
    "Find the total duration of all tracks.": "SELECT SUM(Milliseconds) FROM tracks;",
    "List all customers from Canada.": "SELECT * FROM customers WHERE Country = 'Canada';",
    "How many tracks are there in the album with ID 5?": "SELECT COUNT(*) FROM tracks WHERE AlbumId = 5;",
    "Find the total number of invoices.": "SELECT COUNT(*) FROM invoices;",
    "List all tracks that are longer than 5 minutes.": "SELECT * FROM tracks WHERE Milliseconds > 300000;",
    "Who are the top 5 customers by total purchase?": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM invoices GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    "Which albums are from the year 2000?": "SELECT * FROM albums WHERE strftime('%Y', ReleaseDate) = '2000';",
    "How many employees are there": 'SELECT COUNT(*) FROM "employee"',
}

그런 다음 질문 목록을 사용하여 검색기를 생성하고 대상 SQL 쿼리를 메타데이터로 할당

In [16]:
!pip install tiktoken faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 45.3 MB/s eta 0:00:00


In [17]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

few_shot_docs = [
    Document(page_content=question, metadata={"sql_query": few_shots[question]})
    for question in few_shots.keys()
]
vector_db = FAISS.from_documents(few_shot_docs, embeddings)
retriever = vector_db.as_retriever()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


고유한 사용자 지정 도구를 만들어 'create_sql_agent' 함수에 새 도구로 추가

In [18]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool_description = """
이 도구는 유사한 예시를 이해하여 사용자 질문에 적용하는 데 도움이 됩니다.
이 도구에 입력하는 내용은 사용자 질문이어야 합니다.
"""

retriever_tool = create_retriever_tool(
    retriever, name="sql_get_similar_examples", description=tool_description
)
custom_tool_list = [retriever_tool]

사용 사례를 고려하여 표준 SQL 에이전트 접미사를 조정하여 에이전트 생성

In [21]:
from langchain.agents import AgentType, create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///chinook.db")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

custom_suffix = """
먼저 제가 알고 있는 비슷한 예제를 가져와야 합니다.
예제가 쿼리를 구성하기에 충분하다면 쿼리를 작성할 수 있습니다.
그렇지 않으면 데이터베이스의 테이블을 살펴보고 쿼리할 수 있는 항목을 확인할 수 있습니다.
그런 다음 가장 관련성이 높은 테이블의 스키마를 쿼리해야 합니다.
"""

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    extra_tools=custom_tool_list,
    suffix=custom_suffix,
)

In [22]:
agent.run("How many employees do we have?")



> Entering new SQL Agent Executor chain...

Invoking: `sql_get_similar_examples` with `{'query': 'How many employees do we have?'}`


How many employees are there

Find the total number of invoices.

Who are the top 5 customers by total purchase?

List all customers from Canada.
Invoking: `sql_db_list_tables` with `{}`


albums, artists, customers, employees, genres, invoice_items, invoices, media_types, playlist_track, playlists, tracks
Invoking: `sql_db_schema` with `{'table_names': 'employees'}`



CREATE TABLE employees (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES e

'We have a total of 8 employees.'

- 에이전트는 먼저 `sql_get_similar_examples` 도구를 사용하여 유사한 예제를 검색했음
- 질문이 다른 몇 개의 샷 예제와 매우 유사했기 때문에 에이전트는 표준 툴킷의 다른 툴을 사용할 필요가 없었기 때문에 시간과 토큰 절약